In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from mlxtend.plotting import plot_decision_regions
from scipy.io import arff

In [ ]:
data = arff.loadarff("data/rice.arff")
df = pd.DataFrame(data[0])

In [ ]:
df['Class'] = df['Class'].map({b'Cammeo': 0, b'Osmancik': 1})
X = df.drop(columns=['Class'])
y = df['Class']
X = PCA(n_components=2).fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = [
    SVC(kernel='linear', C=1, degree=5, gamma="auto", probability=True),
    SVC(kernel='rbf', C=1, gamma='scale', probability=True),
    SVC(kernel='poly', C=1, degree=5, gamma="auto", probability=True)
]

In [ ]:
for model in models:
    pipeline = make_pipeline(StandardScaler(), model)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    print(f"SVM ({model.kernel} kernel) Accuracy: {accuracy_score(y_test, y_pred):.2f}")
    print(classification_report(y_test, y_pred))
    
    plot_decision_regions(X=X_train, y=y_train.values, clf=pipeline, legend=2)
    plt.title(f'SVM Decision Region Boundary ({model.kernel} kernel)')
    plt.show()
    
    ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred), display_labels=[0, 1]).plot(cmap='Blues')
    plt.show()
    
    y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], 'k--', label='No Skill')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve for Rice Classification')
    plt.legend()
    plt.show()